<a href="https://colab.research.google.com/github/anmorgan24/COVID-Weather-Capstone/blob/main/testing_yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://cdn.comet.ml/img/notebook_logo.png">

# 🏋 Custom Training with YOLOv5 + Comet Artifacts
___

Comet makes tracking your custom model and dataset easier than ever, so you can focus on more important tasks at hand. In this notebook, we'll demonstrate how to use Comet with Yolov5 to train a dataset Artifact and track model metrics, inspect model outputs, restart interrupted runs, and more!

Comet Artifacts allow you to version  any type of data asset, as well as track their lineages through a project. For this tutorial, we'll be using the publicly-available `penguin-detection-dataset`, but feel free to create your own, or upload your trained yolov5 model as an Artifact at the end of this tutorial! To learn more about how to create your own Artifacts, check out [this tutorial](https://colab.research.google.com/drive/1eOL6mJGzM7F1hPd3Wu4VsLZp8SjMrLbp?authuser=1#scrollTo=zeUyqpKXLLmt)!


# 🔩 Install Requirements & Set Up

___

In [ ]:
%pip install comet_ml --quiet

The following cell will ask for your Comet API key. You can find it at the link provided, or, if you don't have account yet, [create one here for free](https://www.comet.com/signup?utm_source=yolov5&utm_medium=partner&utm_campaign=online_partner_yolov5_integration).

In [ ]:
import comet_ml
comet_ml.init(project_name='penguin-detection',
              workspace="anmorgan24")

In [ ]:
!git clone https://github.com/DN6/yolov5 && cd yolov5 && git checkout feature-comet-integration
%cd yolov5 
%pip install -qr requirements.txt

In [ ]:
import torch
import utils
import os
display = utils.notebook_init()  # checks
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Here, we'll create a `yaml` file for our penguins dataset that points to our Comet Artifact. If you're using your own Artifact, the `path` should follow the structure: `"comet://<workspace-name>/<dataset-name>:<version or alias>"`.



In [ ]:
import yaml

penguins_file = ({
  "names": {
    0: "penguin"
  },
  "nc": 1,
  "path": "comet://team-comet-ml/penguin-detection-dataset:latest",
  "test": "/test/images",
  "train": "/train/images",
  "val": "/valid/images"
})

with open(r'penguins.yaml', 'w') as file:
    documents = yaml.dump(penguins_file, file)

# ⭐ Log Automatically With Comet

With the Yolov5 integration, **Comet automatically logs each of the following, straight out-of-the-box, and without any additional code:**

#### **Metrics:**
- Box Loss, Object Loss, Classification Loss for the training and validation data
- mAP_0.5, mAP_0.5:0.95 metrics for the validation data.
- Precision and Recall for the validation data

#### **Parameters:**
- Model Hyperparameters
- All parameters passed through the command line options
- Confusion Matrix of the model predictions on the validation data

#### **Visualiztions:**
- Plots for the PR and F1 curves across all classes
- Correlogram of the Class Labels

[![auto-logging.gif](https://s4.gifyu.com/images/auto-logging.gif)](https://www.comet.com/site/)

Later on we'll explore even more custom logging capabailites, but it looks like we're off to a great start with Comet's automatic logging functionalities!


# 🦾 Train YOLOv5 on a Comet Artifact
____


Here we're able to pass a number of arguments to initiate our training run:
- **`img`:** defines original input image size; the model will perform all necessary pre-processing, including resizing, as well as scaling the pixel range to 0-1.
- **`batch`:** defines the batch size.
- **`epochs`:** defines the number of training epochs.  
- **`data`:** specifies the name of the `yaml` file (created above) that will point to the Comet Artifact containing our dataset. 
- **`cache`:** provides the option to cache images for faster training.
- **`weights`:** specifies the path to the saved weights you'd like to start our transfer learning from. For this tutorial, we will choose the weights corresponding to the "small" `yolov5s` checkpoint.

Additionally, we can log model predictions (and their associated images) by passing the **`bbox_interval`** command line argument. These bounding box predictions can then be visualized using Comet's Object Detection Custom Panel.

Here is an [example project using the Panel](https://www.comet.com/examples/comet-example-yolov5?shareable=YcwMiJaZSXfcEXpGOHDD12vA1&ref=yolov5&utm_source=yolov5&utm_medium=affilliate&utm_campaign=yolov5_comet_integration), or see the gif below:





In [ ]:
python tools/benchmarking/benchmark.py \
--config <relative-or-absolute-path>/<paramfile>.yaml

In [ ]:
!python train.py \
--img 640 \
--batch 16 \
--epochs 3 \
--data ./penguins.yaml \
--weights yolov5s.pt \
--bbox_interval 1

[![penguins_yolov5-2.gif](https://s5.gifyu.com/images/penguins_yolov5-2.gif)](https://www.comet.com/site/)

# 🛠 Configure Custom Comet Logging


Comet's custom logging capabilities don't end there! Track and visualize model metrics in real time, save your hyperparameters, datasets, and model checkpoints, and visualize your model predictions with [Comet Custom Panels](https://www.comet.com/docs/v2/guides/comet-dashboard/code-panels/about-panels/?utm_source=colab&utm_medium=referral&utm_campaign=yolov5_comet_integration_launch)! Here are a few more examples of what we can do with just a simple command line flag or environment variable:



## Uploading a Dataset to Comet Artifacts


If you would like to store your data using [Comet Artifacts](https://www.comet.com/docs/v2/guides/data-management/using-artifacts/#learn-more?ref=yolov5&utm_source=yolov5&utm_medium=affilliate&utm_campaign=yolov5_comet_integration), you can do so using the `upload_dataset` flag.

The dataset be organized in the way described in the [YOLOv5 documentation](https://docs.ultralytics.com/tutorials/train-custom-datasets/#3-organize-directories). The dataset config `yaml` file must follow the same format as that of the `coco128.yaml` file (built-in to Yolov5) or the `penguins.yaml` file (shown above).

In [ ]:
!python train.py \
--img 640 \
--batch 16 \
--epochs 5 \
--data ./penguins.yaml \
--weights yolov5s.pt \
--upload_dataset "train"

### Using a saved Artifact

If you would like to use a different dataset from Comet Artifacts, or if you would like to be able to try different datasets on the same model, simply set the `path` variable in your dataset `yaml` file to point to the following Artifact resource URL:

```
# contents of artifact.yaml file
path:"comet://<workspace name>/<artifact name>:<artifact version or alias>"
```
Then pass your file to your training script with this command:

In [ ]:
!echo "path: \"comet://<workspace name>/<artifactnam>:<artifact version or alias>\"" >> /content/yolov5/data/artifact.yaml

In [ ]:
!python train.py \
--img 640 \
--batch 16 \
--epochs 5 \
--data /content/yolov5/data/artifact.yaml \
--weights yolov5s.pt \

## Logging Checkpoints with Comet

Logging Models to Comet is disabled by default, but to enable it, just pass the `save-period` argument to the training script. This will save the
logged checkpoints to Comet based on the interval value provided to `save-period`.

Find yor checkpoints in the [Assets and Artifacts](https://www.comet.com/examples/comet-example-yolov5/a0e29e0e9b984e4a822db2a62d0cb357?assetId=7df85817e69b4997a803f947334155e6&assetPath=models%2Cyolov5&experiment-tab=assets) tab in your Experiment.

In [ ]:
!python train.py \
--img 640 \
--batch 16 \
--epochs 5 \
--data coco128.yaml \
--weights yolov5s.pt \
--save-period 1 

You can find the uploaded dataset in the Artifacts tab in your Comet workspace and you can preview the data directly in the Comet UI. Artifacts are versioned and also support adding metadata. Comet will automatically log this metadata from your dataset `yaml` file.

### Controlling the number of Prediction Images logged to Comet

When logging predictions from YOLOv5, Comet will log the images associated with each set of predictions. By default a maximum of 100 validation images are logged. You can increase or decrease this number using the `COMET_MAX_IMAGE_UPLOADS` environment variable.


In [ ]:
!env COMET_MAX_IMAGE_UPLOADS=200 python train.py \
--img 640 \
--batch 16 \
--epochs 5 \
--data coco128.yaml \
--weights yolov5s.pt \
--bbox_interval 1

### Logging Class Level Metrics

Use the `COMET_LOG_PER_CLASS_METRICS` environment variable to log mAP, precision, recall, f1 for each class.

In [ ]:
!env COMET_LOG_PER_CLASS_METRICS=true python train.py \
--img 640 \
--batch 16 \
--epochs 5 \
--data coco128.yaml \
--weights yolov5s.pt 

## Resuming a Training Run

If your training run is interrupted for any reason, e.g. disrupted internet connection, you can resume the run using the `resume` flag and the Comet Run Path.

The Run Path has the following format `comet://<your workspace name>/<your project name>/<experiment id>`.

This will restore the run to its state before the interruption, which includes restoring the  model from a checkpoint, restoring all hyperparameters and training arguments and downloading Comet dataset Artifacts if they were used in the original run. The resumed run will continue logging to the existing Experiment in the Comet UI

In [ ]:
!python train.py \
--resume "comet://<your run path>"